In [41]:
import pandas as pd
import numpy as np


In [42]:
file_path = 'dongjak_with_crosswalk_accident_count.xlsx'

In [43]:
df = pd.read_excel(file_path)
print(df.columns) 
# 사용할 컬럼 : '사건수','횡단보도관리번호','녹색신호시간'

Index(['Unnamed: 0', 'acc_x', 'acc_y', '시도명', '시군구명', '도로명', '소재지도로명주소',
       '횡단보도관리번호', '횡단보도종류', '자전거횡단도겸용여부', '고원식적용여부', 'grsXCrd', 'grsYCrd',
       '차로수', '횡단보도폭', '횡단보도연장', '보행자신호등유무', '보행자작동신호기유무', '음향신호기설치여부',
       '녹색신호시간', '적색신호시간', '교통섬유무', '보도턱낮춤여부', '점자블록유무', '집중조명시설유무', '관리기관명',
       '관리기관전화번호', '데이터기준일자', '제공기관코드', '제공기관명', '사건수'],
      dtype='object')


In [44]:
filterd_df = pd.read_excel(file_path, sheet_name="Sheet1", header=0, usecols=['사건수','횡단보도관리번호','녹색신호시간'])
ml_df = filterd_df.rename(columns={
    '횡단보도관리번호':'crosswalk_id',
    '사건수':'acc_count',
    '녹색신호시간':'green_signal_time'
})

In [45]:
print("=== 데이터 현황 ===")
print(f"\n전체 데이터: {len(ml_df)}행")
print(f"녹색신호시간 결측값: {ml_df['green_signal_time'].isnull().sum()}개")
print(f"고령사 교통사고 발생 사건수 결측값: {ml_df['acc_count'].isnull().sum()}개")


=== 데이터 현황 ===

전체 데이터: 408행
녹색신호시간 결측값: 281개
고령사 교통사고 발생 사건수 결측값: 0개


In [46]:
# 결측치 제거
ml_df = ml_df.dropna()
print("=== 데이터 현황 ===")
print(f"\n 결측치 제거 후 데이터: {len(ml_df)}행")
print(ml_df.head(10))

=== 데이터 현황 ===

 결측치 제거 후 데이터: 127행
     crosswalk_id  green_signal_time  acc_count
0   06-0000043750               33.0          2
2   06-0000049199               27.0          2
5   06-0000043760               31.0          1
7   06-0000006143               32.0          1
25  06-0000006442               28.0          3
26  06-0000038747               30.0          1
27  06-0000006443               25.0          1
30  06-0000006441               30.0          1
34  06-0000006448               27.0          3
40  06-0000031765               26.0          3


In [47]:
# 사고발생 여부 벡터화
ml_df['is_accident'] = (ml_df['acc_count']>0).astype(int)
print(ml_df.head(10))

     crosswalk_id  green_signal_time  acc_count  is_accident
0   06-0000043750               33.0          2            1
2   06-0000049199               27.0          2            1
5   06-0000043760               31.0          1            1
7   06-0000006143               32.0          1            1
25  06-0000006442               28.0          3            1
26  06-0000038747               30.0          1            1
27  06-0000006443               25.0          1            1
30  06-0000006441               30.0          1            1
34  06-0000006448               27.0          3            1
40  06-0000031765               26.0          3            1


In [48]:
%pip install matplotlib
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 로지스틱 회귀모델 분석

In [68]:
import statsmodels.api as sm

In [69]:
X = ml_df['green_signal_time']
y = ml_df['is_accident']

In [70]:
logit_model = sm.Logit(y, X).fit(maxiter=100)

         Current function value: -0.000000
         Iterations: 100


/Users/yeonsu/Desktop/hackerton_backend/.venv/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Users/yeonsu/Desktop/hackerton_backend/.venv/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Users/yeonsu/Desktop/hackerton_backend/.venv/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/Users/yeonsu/Desktop/hackerton_backend/.venv/lib/python3.13/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or predi

LinAlgError: Singular matrix

In [65]:
# 교차검증 (K-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)
scores = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')

In [66]:
# 랜덤포레스트 모델 정확도
print(f'RandomForestClassifier cv Accuray: {scores.mean():.3f}(std: {scores.std():.3f})')

RandomForestClassifier cv Accuray: 1.000(std: 0.000)


## 정확도가 100%.. 문제가 있다.